In [ ]:
!git clone https://github.com/t-davidson/hate-speech-and-offensive-language.git

Cloning into 'hate-speech-and-offensive-language'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 44 (delta 6), reused 2 (delta 0), pack-reused 32 (from 1)
Receiving objects: 100% (44/44), 3.56 MiB | 25.13 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv("hate-speech-and-offensive-language/data/labeled_data.csv")

# Display first few rows
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [ ]:
#preprocessing the data
import torch
from transformers import BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text data
df['tokenized'] = df['tweet'].apply(lambda x: tokenizer(x, padding='max_length', max_length=128, truncation=True, return_tensors="pt"))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
#creating the data set for torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd # Import pandas
import os # Import the os module
from transformers import BertTokenizer # Import BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class HateSpeechDataset(Dataset):
    def __init__(self, dataframe):
        # Tokenize the text data within the __init__ method
        dataframe['tokenized'] = dataframe['tweet'].apply(lambda x: tokenizer(x, padding='max_length', max_length=128, truncation=True, return_tensors="pt"))
        self.texts = dataframe['tokenized']
        self.labels = dataframe['class'].values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx], torch.tensor(self.labels[idx])

# Get the current working directory
current_directory = os.getcwd()

# Construct the absolute path to the CSV file
file_path = os.path.join(current_directory, "hate-speech-and-offensive-language/data/labeled_data.csv")

# Load the dataset using the absolute path # Load the dataframe here
df = pd.read_csv(file_path)

# Create dataset and dataloader
dataset = HateSpeechDataset(df) # Now the df passed to HateSpeechDataset has the 'tokenized' column
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
#fine-tuned bert for hate speech classification
from transformers import BertForSequenceClassification

# Load BERT model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)  # 3 classes

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
!pip install transformers --upgrade

In [ ]:
from tqdm import tqdm # Import tqdm for progress bar
from transformers import AdamW #Importing AdamW here, instead of importing from transformers
from transformers import get_linear_schedule_with_warmup

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# You can also optionally add a learning rate scheduler
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataloader) * epochs)


# Define loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
epochs = 4
for epoch in range(epochs):
    model.train()
    total_loss = 0

    print(f"\nEpoch {epoch+1}/{epochs}")

    progress_bar = tqdm(dataloader, desc=f"Training Epoch {epoch+1}", leave=True)

    for step, batch in enumerate(progress_bar):
        inputs, labels = batch
        inputs = {key: val.squeeze().to(device) for key, val in inputs.items()}
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        # scheduler.step() # Update the learning rate if using a scheduler

        total_loss += loss.item()

        # Update progress bar with loss info
        progress_bar.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1} Completed | Average Loss: {avg_loss:.4f}")

ImportError: cannot import name 'AdamW' from 'transformers' (/usr/local/lib/python3.11/dist-packages/transformers/__init__.py)

In [ ]:
torch.save(model.state_dict(), "hate_speech_model.pth")

In [ ]:
def predict(text):
    model.eval()
    tokens = tokenizer(text, return_tensors="pt", padding='max_length', max_length=128, truncation=True).to(device)
    output = model(**tokens)
    prediction = torch.argmax(output.logits, dim=1).item()

    classes = {0: "Hate Speech", 1: "Offensive Language", 2: "Neutral"}
    return classes[prediction]

# Example
print(predict("shut  up!"))  # Should detect hate speech or offensive language

Offensive Language


In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 111.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.7 MB/s eta 0:00:00


In [ ]:
import torch
import gradio as gr

def predict(text):
    model.eval()
    tokens = tokenizer(text, return_tensors="pt", padding='max_length', max_length=128, truncation=True).to(device)
    output = model(**tokens)
    prediction = torch.argmax(output.logits, dim=1).item()

    classes = {0: "Hate Speech", 1: "Offensive Language", 2: "Neutral"}
    return classes[prediction]

# Creating a Gradio Interface
iface = gr.Interface(fn=predict, inputs="text", outputs="text", title="Hate Speech Detection")
iface.launch(share=True)  # 'share=True' generates a public link

ModuleNotFoundError: No module named 'gradio'